In [ ]:
import pandas as pd
import numpy as np
from util.config import versioned

filter_region = "10"  # Corresponds to wojewodztwo TERYT code

matched_all = versioned.read_parquet("matched.parquet")
# Filter people without any wojewodztwo info
matched_all = matched_all[matched_all["teryt_wojewodztwo"].notnull()]
# Get people with high enough scores
# TODO use the collision chance metric
good = matched_all["overall_score"] > 10.5
# Get people for the given region
local = matched_all["teryt_wojewodztwo"].apply(lambda row: filter_region in row)
local_good = matched_all[good & local]
# Filter out duplicates
def drop_duplicates(df, *cols):
    for col in cols:
        df = df[(~df[col].duplicated()) | df[col].isna()]
    return df

local_good = drop_duplicates(local_good, "krs_name", "pkw_name", "wiki_name")
f"Znaleziono {len(local_good)} osób"

In [ ]:
local_good["krs_name"].nunique(), local_good["pkw_name"].nunique(), local_good["wiki_name"].nunique()

In [ ]:
pd.set_option('display.max_rows', 600)
local_good

In [ ]:
# Write to output
local_output = versioned.get_path("people_lodzkie.csv")
local_good.to_csv(local_output,index=False)